In [1]:
# nugraphrag1 - no csv 
import torch
import networkx as nx
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize GPT-2 Model
def initialize_model():
    """Initialize the tokenizer and GPT-2 model."""
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2", padding_side="left")  # Set padding_side to 'left'
    model = GPT2LMHeadModel.from_pretrained("gpt2").half()  # Load GPT-2 in half-precision
    model.to(device)  # Make sure it's on the correct device

    # Set pad_token (GPT-2 doesn't have one by default, so we use eos_token as pad_token)
    tokenizer.pad_token = tokenizer.eos_token
    
    return tokenizer, model

# Step 1: Create a Knowledge Graph (Example with NetworkX)
def create_knowledge_graph():
    """Create a sample knowledge graph."""
    G = nx.Graph()
    
    # Example nodes (entities) and edges (relationships)
    G.add_node("inner_beauty")
    G.add_node("mindfulness")
    G.add_node("well_being")
    
    G.add_edge("inner_beauty", "mindfulness", relationship="promotes")
    G.add_edge("mindfulness", "well_being", relationship="improves")
    G.add_edge("inner_beauty", "well_being", relationship="enhances")
    
    return G

# Step 2: Retrieve Knowledge from Graph
def retrieve_from_graph(graph, query):
    """Retrieve relevant information from the knowledge graph based on the input query."""
    relevant_nodes = []
    relevant_edges = []
    
    # Simple logic to find relevant nodes (could be improved with NLP techniques)
    if "beauty" in query:
        relevant_nodes.append("inner_beauty")
        relevant_nodes.append("well_being")
    if "mindfulness" in query:
        relevant_nodes.append("mindfulness")
    
    # Retrieve edges based on relevant nodes
    for node in relevant_nodes:
        neighbors = list(graph.neighbors(node))
        for neighbor in neighbors:
            edge = graph.get_edge_data(node, neighbor)
            relevant_edges.append((node, neighbor, edge))
    
    return relevant_nodes, relevant_edges

# Step 3: Use Graph Knowledge to Generate Context-Aware Responses with GPT-2
def generate_with_graph_knowledge(query, model, tokenizer, graph):
    """Generate a response based on a query and relevant knowledge from the graph."""
    # Retrieve relevant nodes and edges
    relevant_nodes, relevant_edges = retrieve_from_graph(graph, query)
    
    # Format the retrieved knowledge into a context string
    context = f"Query: {query}\n"
    context += f"Relevant Knowledge:\n"
    
    for node in relevant_nodes:
        context += f" - {node}\n"
    
    for edge in relevant_edges:
        context += f" - {edge[0]} {edge[2]['relationship']} {edge[1]}\n"
    
    # Use GPT-2 to generate a response with the enriched context
    inputs = tokenizer(context, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Generate the response
    outputs = model.generate(
        **inputs, 
        max_new_tokens=200, 
        num_return_sequences=1, 
        pad_token_id=tokenizer.pad_token_id
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Save and Load Knowledge Graph
def save_graph(graph, filepath="knowledge_graph.pkl"):
    """Save the knowledge graph to a file."""
    import pickle
    with open(filepath, "wb") as f:
        pickle.dump(graph, f)

def load_graph(filepath="knowledge_graph.pkl"):
    """Load the knowledge graph from a file."""
    import pickle
    with open(filepath, "rb") as f:
        return pickle.load(f)

# Main Processing Function
def main():
    """Main processing loop."""
    # Filepath to save/load the graph
    save_path = "knowledge_graph.pkl"
    
    # Load or create the knowledge graph
    try:
        graph = load_graph(save_path)
        print("Knowledge graph loaded successfully.")
    except FileNotFoundError:
        graph = create_knowledge_graph()
        save_graph(graph, save_path)
        print("Knowledge graph created and saved.")
    
    # Initialize the model
    tokenizer, model = initialize_model()

    # Example query
    query = "How can mindfulness improve well-being and inner beauty?"

    # Generate a response using the graph knowledge
    response = generate_with_graph_knowledge(query, model, tokenizer, graph)

    # Return the generated response
    return response

# Example Execution
if __name__ == "__main__":
    try:
        output = main()  # Process the query and generate a response
        print(output)
    except Exception as e:
        print(f"An error occurred: {e}")


Using device: cpu
Knowledge graph loaded successfully.


KeyboardInterrupt: 